Detección de caras con webcam

In [1]:
import cv2
import time
#from deepface import DeepFace
from retinaface import RetinaFace


Deepface

Deepface crea directorios para descargar los modelos, cuidado si tienes el disco bastante lleno. Es posible configurar la ruta, en mi caso uso  E:\RUNNERS_code\code\DeepFace, tras definir la ruta a través de la variable de entorno DEEPFACE_HOME

pip install retina-face o pip install deepface

In [1]:
import cv2
import time
#from deepface import DeepFace
from retinaface import RetinaFace

# Webcam connection
cap = cv2.VideoCapture(0)
# Check for other cameras
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

#Set camera resolution
cap.set(3,640)
cap.set(4,480)

glasses_count = 0
glasses = []

for i in range(0,7):
    g = cv2.imread('./images/gafas/' + str(i) + '.png',cv2.IMREAD_UNCHANGED)
    width_resized = int(g.shape[1] * 60 / 100)
    height_resized = int(g.shape[0] * 60 / 100)
    dim = (width_resized, height_resized)
    mask = cv2.resize(g, dim)
    glasses.append(mask)

def setMask(mask, xCoord, yCoord):
    rows, cols, _ = mask.shape

    if yCoord - rows > 0 and xCoord - int(cols / 2) > 0:
        # Asegurarse de que mask tenga un canal alfa
        if mask.shape[2] == 4:
            
            # Definir la región de superposición
            y1, y2 = yCoord - int(rows / 2), yCoord + int(rows / 2)
            x1, x2 = xCoord - int(cols / 2), xCoord + int(cols / 2)

            # Crear la máscara alfa
            alpha_s = mask[:, :, 3] / 255.0
            alpha_l = 1.0 - alpha_s

            # Superponer la imagen con transparencia en frame2
            for c in range(0, 3):
                frame2[y1:y2, x1:x2, c] = (alpha_s * mask[:, :, c] + alpha_l * frame2[y1:y2, x1:x2, c])

# print(mask.shape)
# print(mask.shape[1])
# cv2.imshow('Test', mask)


while True:
    # Get frame
    #t = time.time()
    ret, frame = cap.read()

    # Añade el canal alfa a frame
    frame2 = cv2.cvtColor(frame, cv2.COLOR_BGR2BGRA)

    #Deepface detecta caras y se queda con primera
    #face = DeepFace.detectFace(img_path = frame, 
    #    target_size = (224, 224), 
    #    detector_backend = backends[4]
    #)

    #Detectar múltiples (https://sefiks.com/2021/04/27/deep-face-detection-with-retinaface-in-python/)
    #Devuelve un dict
    faces = RetinaFace.detect_faces(frame)
    #Si hay detecciones
    if len(faces) > 0:
        #Dibuja contenedor y elementos faciales
        for idx in range(1,len(faces)+1):
            id = 'face_' + str(idx)
            #Muestra información de caras
            #print(faces[id])
            #Accediendo a contenedor de la cara
            facial_area = faces[id]['facial_area']
            #Accediendo a datos de los elementos faciales
            landmarks = faces[id]['landmarks']
            #Debug
            #print(facial_area)
            #print(landmarks)
            
            #Contenedor
            cv2.rectangle(frame, (facial_area[2], facial_area[3])
            , (facial_area[0], facial_area[1]), (255, 255, 255), 1)
            
            xCoord_ojos = round((int(landmarks["left_eye"][0]) + int(landmarks["right_eye"][0])) / 2)
            yCoord_ojos = int(landmarks["left_eye"][1])
            # print("xCoord_ojos = " + str(xCoord_ojos))
            # print("yCoord_ojos = " + str(yCoord_ojos))

            mask = glasses[glasses_count]
            xCoord_ojos = round((int(landmarks["left_eye"][0]) + int(landmarks["right_eye"][0])) / 2)
            yCoord_ojos = int(landmarks["left_eye"][1])
            setMask(mask, xCoord_ojos, yCoord_ojos)          
     
    #Muestra imagen
    cv2.imshow('Cam', frame2)
    
    #Esperar por tecla o seguir
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    elif tec & 0xFF == ord('d'):
        print(len(glasses))
        if(glasses_count >= len(glasses) - 1):
            glasses_count = 0
        else:
            glasses_count += 1   
    elif tec & 0xFF == ord('f'):
        print(len(glasses))
        if(glasses_count == 0):
            glasses_count = len(glasses) - 1
        else:
            glasses_count -= 1   
    
#Cerrar ventanas y cámara
cap.release()
cv2.destroyAllWindows()

Camera 0
7
7
7
Entro
7
7
7
